In [ ]:
%pip install -U langchain-anthropic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_core-0.3.66-py3-none-any.whl.metadata (5.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.10
    Uninstalling langsmith-0.2.10:
      Successfully uninstalled langsmith-0.2.10
  Attempting uninstall: anthropic
    Found existing installation: anthropic 0.45.2
    Uninstalling anthropic-0.45.2:
      Successfully uninstalled anthropic-0.45.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
  Attempting uninstall: langchain-anthropic
    Found existing installation: langchain-anthropic 0.3.6
    Uninstalling langchain-anthropic-0.3.6:
      Successfully uninstalled langchain-anthropic-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

UsageError: Line magic function `%export` not found.


In [1]:
import csv
import json
from datasets import Dataset
from chromadb.utils import embedding_functions

import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
import pandas as pd
from ragas import evaluate
from pipeline.llm_utils import LLMApiClient
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas import EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from copy import deepcopy
from langchain_anthropic import ChatAnthropic

In [10]:
human_ds = Dataset.from_csv("Plan Net Requirements(1.1.csv")

In [7]:
dlist[0]

{'Section #': '5.1.2',
 'Link': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/STU1.1/implementation.html#privacy-considerations-1',
 'Section Title': 'Privacy Considerations',
 'Requirement': 'A conformant Plan-Net service SHALL NOT require a directory mobile application to send consumer identifying information in order to query content.',
 'Conformance': 'SHALL NOT',
 'Target': 'Server',
 'Requirement Id (See 1.1.0 Tests tab)': 'S-NoPII',
 'Tested?': 'No',
 'Tested Response Details': 'Untestable',
 'Testable? (see Testability and Test Approach Tab)': 'No',
 'Notes / Questions': 'Can this be approximated by requiring that authentication details not be sent?',
 'Target Draft Scope': 'No',
 'Status': 'NA',
 'JIRA links': ''}

In [20]:
with open("../internal_onclaive/onclaive/reqs_extraction/experiment_output/rag-lite-plannet-reqs-processed_v0.json") as f:
    ragreqs = json.load(f)
# with open("../internal_onclaive/onclaive/reqs_extraction/experiment_output/rag-lite-plannet-reqs.json") as f:
#     ragreqs = json.load(f)

In [23]:
ragreqs[0].keys()

dict_keys(['file_name', 'chunk_index', 'total_chunks', 'chunk', 'response', 'llm_response', 'selected_examples', 'processed_requirements'])

In [31]:
i = 0
for entry in ragreqs:
    if 'processed_requirements' in entry:
        i += 1
print(i, len(ragreqs))

138 162


In [26]:
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name="all-mpnet-base-v2")

In [76]:
rag_strings = []
for entry in ragreqs:
    if 'processed_requirements' in entry:
        for req in entry['processed_requirements']:
            rag_strings.append({'requirement_text': req['Requirement*'], 'requirement_object': req, 'chunk': entry['chunk']})

rag_ds = Dataset.from_list(rag_strings)

In [ ]:
vectors = ef.embed_with_retries(rag_strings.keys())

In [35]:
human_vectors = ef.embed_with_retries(human_ds['Requirement'])

In [37]:
mat = pairwise_distances(vectors, Y=human_vectors, metric='cosine')

In [38]:
mat.shape

(575, 40)

In [39]:
mat

array([[0.691572  , 0.6874276 , 0.72738445, ..., 0.7919129 , 0.7919129 ,
        0.70331466],
       [0.724698  , 0.75139505, 0.5967315 , ..., 0.69035614, 0.69035614,
        0.64132285],
       [0.70380485, 0.75014585, 0.63428867, ..., 0.6823095 , 0.6823095 ,
        0.62140566],
       ...,
       [0.8861651 , 0.90680355, 0.79491925, ..., 0.93528754, 0.93528754,
        0.8136785 ],
       [0.7201339 , 0.72039175, 0.6750707 , ..., 0.64340734, 0.64340734,
        0.80129313],
       [0.8521931 , 0.90468967, 0.913733  , ..., 0.8979242 , 0.8979242 ,
        0.81516   ]], dtype=float32)

In [41]:
df = pd.DataFrame(mat)

In [55]:
np.argmax(mat, axis=0)

array([569, 569,  88,  88, 404, 569, 481,  88, 569, 574, 461, 574, 211,
        57, 115, 115, 542,  88,  86,  86, 409, 409, 574, 421, 189, 574,
       530, 463, 556, 574, 549, 387, 387, 387, 387, 387, 112, 111, 111,
       305])

In [ ]:
[list(rag_strings.keys())[x] for x in np.argmax(mat, axis=0)]

40

In [79]:
rag_ds[0]['requirement_object']

{'Actor*': 'Server',
 'Conformance*': 'SHALL',
 'Grouping': 'Terminology',
 'Notes': None,
 'Planning To Test Details': None,
 'Planning To Test?': 'Yes',
 'Questions': None,
 'Requirement*': "Server SHALL support the CodeSystem 'DeliveryMethodCS' with URL 'http://hl7.org/fhir/us/davinci-pdex-plan-net/CodeSystem/DeliveryMethodCS'",
 'Simulation Approach': 'SIMULATED: Inferno will include this CodeSystem in its terminology validation.',
 'Test Plan': 'Verify that the server supports the DeliveryMethodCS CodeSystem by validating a resource that uses a code from this CodeSystem.',
 'Test name': None,
 'Verifiable?': 'Yes',
 'client testing simulation implementation group': None}

In [75]:
human_ds[0]

{'Section #': '5.1.2',
 'Link': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/STU1.1/implementation.html#privacy-considerations-1',
 'Section Title': 'Privacy Considerations',
 'Requirement': 'A conformant Plan-Net service SHALL NOT require a directory mobile application to send consumer identifying information in order to query content.',
 'Conformance': 'SHALL NOT',
 'Target': 'Server',
 'Requirement Id (See 1.1.0 Tests tab)': 'S-NoPII',
 'Tested?': 'No',
 'Tested Response Details': 'Untestable',
 'Testable? (see Testability and Test Approach Tab)': 'No',
 'Notes / Questions': 'Can this be approximated by requiring that authentication details not be sent?',
 'Target Draft Scope': 'No',
 'Status': None,
 'JIRA links': None}

In [ ]:
# obj = list(zip(human_ds.to_list(), [dict(rag_ds[int(x)]) for x in np.argmax(mat, axis=0)]))

# with open("temp_pairs.json", "w+") as f:
#     json.dump(obj, f)

In [4]:
with open('temp_pairs.json') as f:
    pairs = json.load(f)

In [5]:
# pairs = zip(human_ds, [rag_ds[int(x)] for x in np.argmax(mat, axis=0)])
dataset = []

PROMPT_2_PATH = "prompts/requirement_extraction_rag.txt"

with open(PROMPT_2_PATH, 'r') as f:
    PROMPT_2 = f.read()

for human,rag in pairs:
    dataset.append(
        {
            "user_input":PROMPT_2,
            "retrieved_contexts":[str(rag['chunk'])],
            "response":str(rag['requirement_object']),
            "reference":str(human)
        }
    )

In [3]:
model = ChatAnthropic(model_name="claude-3-5-sonnet-20241022")

In [7]:
evald = EvaluationDataset.from_list(dataset)

result = evaluate(dataset=evald, metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()], llm=LangchainLLMWrapper(model))

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Exception raised in Job[8]: TimeoutError()
Exception raised in Job[28]: TimeoutError()
Exception raised in Job[49]: TimeoutError()
Exception raised in Job[66]: TimeoutError()
Exception raised in Job[73]: TimeoutError()
Exception raised in Job[75]: TimeoutError()
Exception raised in Job[79]: TimeoutError()
Exception raised in Job[108]: TimeoutError()
Exception raised in Job[111]: TimeoutError()


In [10]:
df = result.to_pandas()

In [17]:
a,b = zip(*list(df.iloc[:,2:4].iterrows()))

In [21]:
print(b[0].response)
print(b[0].reference)

{'Actor*': 'Server', 'Conformance*': 'SHALL', 'Grouping': 'Search', 'Notes': None, 'Planning To Test Details': None, 'Planning To Test?': 'Yes', 'Questions': None, 'Requirement*': "The 'address' search parameter for Location resources SHALL support multiple 'AND' instances of the parameter.", 'Simulation Approach': "SIMULATED: Inferno will implement support for multiple 'AND' instances of the parameter in its Location search endpoint.", 'Test Plan': "Execute a search against the server's Location endpoint using multiple instances of the 'address' parameter, verifying that the server returns results matching all of the provided values.", 'Test name': None, 'Verifiable?': 'Yes', 'client testing simulation implementation group': 'Search'}
{'Section #': '5.1.2', 'Link': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/STU1.1/implementation.html#privacy-considerations-1', 'Section Title': 'Privacy Considerations', 'Requirement': 'A conformant Plan-Net service SHALL NOT require a directory mobi

In [99]:
LangchainLLMWrapper(clients.clients['claude']).generate_text("test input")

AttributeError: 'Anthropic' object has no attribute 'generate_prompt'

In [73]:
list(rag_strings.values())[0]

{'Requirement*': "Server SHALL support the CodeSystem 'DeliveryMethodCS' with URL 'http://hl7.org/fhir/us/davinci-pdex-plan-net/CodeSystem/DeliveryMethodCS'",
 'Conformance*': 'SHALL',
 'Actor*': 'Server',
 'Verifiable?': 'Yes',
 'Planning To Test?': 'Yes',
 'Grouping': 'Terminology',
 'Test Plan': 'Verify that the server supports the DeliveryMethodCS CodeSystem by validating a resource that uses a code from this CodeSystem.',
 'Simulation Approach': 'SIMULATED: Inferno will include this CodeSystem in its terminology validation.'}

In [22]:
df.to_csv("20250627_Claude_RAGAS_Requirement_generation_eval_v0.csv")